In [1]:
import torch

import header_code
import logging
torch._logging.set_logs(dynamo=logging.DEBUG)

# Skipped Functions

**Summary:**
- Sometimes, `torch.compile` completely gives up compiling a function and runs it eagerly instead,
  resulting in potentially lost optimization opportunities.
- There are ways to work around skipped functions in order to re-enable tracing around the problematic code.

Sometimes, `torch.compile` with `fullgraph=False` is unable to resume tracing when encountering a graph break
or other compiler error. In many of these cases, `torch.compile` will skip compiling the function entirely and run it eagerly.

Note that the skip is only applied to the current function and NOT any nested function calls.
`torch.compile` will still attempt to compile nested calls.

<!-- TODO: fix logging for skipped functions. -->

In [2]:
def inner1(x):
    return x + 1
def inner2(x):
    return x + 2
@torch.compile
def fn(x):
    x = inner1(x)
    torch._dynamo.skip_frame()
    x = inner2(x)
fn(torch.randn(3))

ChromiumEventLogger initialized with id 806b0fe4-f6a5-4574-9ab4-6bd1d0733e1c
torchdynamo start compiling fn C:\Users\Aditya\AppData\Local\Temp\ipykernel_4800\2126697152.py:5, stack (elided 5 frames):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\dev\pytorch\venv\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\dev\pytorch\venv\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\dev\pytorch\venv\Lib\site-packages\ipykernel\kernelapp.py", line 758, in start
    self.io_loop.start()
  File "C:\dev\pytorch\venv\Lib\site-packages\tornado\platform\asyncio.py", line 211, in start
    self.asyncio_loop.run_forever()
  File "C:\Users\Aditya\AppData\Local\Programs\Python\Python313\Lib\asyncio\base_events.py", line 683, in run_forever
    self._run_once()
  File "C:\Users\Aditya\AppData\Local\Programs\Python\Python313

torchdynamo start compiling fn C:\Users\Aditya\AppData\Local\Temp\ipykernel_4800\2126697152.py:5, stack (elided 5 frames):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\dev\pytorch\venv\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\dev\pytorch\venv\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\dev\pytorch\venv\Lib\site-packages\ipykernel\kernelapp.py", line 758, in start
    self.io_loop.start()
  File "C:\dev\pytorch\venv\Lib\site-packages\tornado\platform\asyncio.py", line 211, in start
    self.asyncio_loop.run_forever()
  File "C:\Users\Aditya\AppData\Local\Programs\Python\Python313\Lib\asyncio\base_events.py", line 683, in run_forever
    self._run_once()
  File "C:\Users\Aditya\AppData\Local\Programs\Python\Python313\Lib\asyncio\base_events.py", line 2050, in _run_once
    handle._run()
  Fil

Step 1: torchdynamo start tracing fn C:\Users\Aditya\AppData\Local\Temp\ipykernel_4800\2126697152.py:5


create_env


TRACE starts_line C:\Users\Aditya\AppData\Local\Temp\ipykernel_4800\2126697152.py:5 in fn
    @torch.compile


...

Guard eval latency = 216.10 us


put_code_state: no cache key, skipping


run_gc_after_compile: running gc


skipping: remove (reason: in skipfiles, file: C:\dev\pytorch\venv\Lib\site-packages\torch\utils\weak.py)


skipping: __hash__ (reason: in skipfiles, file: C:\dev\pytorch\venv\Lib\site-packages\torch\utils\weak.py)


In the above example, `torch.compile` will trace `fn` (including `inner1`) up until the `skip_frame`.
Then `fn` is skipped and run eagerly - `inner1` and `inner2` are compiled when they are called.

Skipping functions may result in lost optimization opportunities,
so it is important to check if code you want compiled is being skipped, and if so, to work around the skip.

## Graph Break in a Loop

`torch.compile` cannot resume tracing if a graph break occurs in a loop:

In [3]:
@torch.compile
def fn(x):
    for i in range(5):
        x = x + 1
        if i == 3:
            torch._dynamo.graph_break()
    return x
fn(torch.randn(3))

torchdynamo start compiling fn C:\Users\Aditya\AppData\Local\Temp\ipykernel_4800\2044822433.py:1, stack (elided 5 frames):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\dev\pytorch\venv\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\dev\pytorch\venv\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\dev\pytorch\venv\Lib\site-packages\ipykernel\kernelapp.py", line 758, in start
    self.io_loop.start()
  File "C:\dev\pytorch\venv\Lib\site-packages\tornado\platform\asyncio.py", line 211, in start
    self.asyncio_loop.run_forever()
  File "C:\Users\Aditya\AppData\Local\Programs\Python\Python313\Lib\asyncio\base_events.py", line 683, in run_forever
    self._run_once()
  File "C:\Users\Aditya\AppData\Local\Programs\Python\Python313\Lib\asyncio\base_events.py", line 2050, in _run_once
    handle._run()
  Fil

Step 1: torchdynamo start tracing fn C:\Users\Aditya\AppData\Local\Temp\ipykernel_4800\2044822433.py:1


create_env


TRACE starts_line C:\Users\Aditya\AppData\Local\Temp\ipykernel_4800\2044822433.py:1 in fn
    @torch.compile


TRACE RESUME 0 []


...

Skipping frame Skipping frame because there is a graph break in a for/while loop
<FrameSummary file C:\Users\Aditya\AppData\Local\Temp\ipykernel_4800\2044822433.py, line 6 in fn> fn                 C:\Users\Aditya\AppData\Local\Temp\ipykernel_4800\2044822433.py 1


put_code_state: no cache key, skipping


run_gc_after_compile: running gc


skipping: graph_break (reason: in skipfiles, file: C:\dev\pytorch\venv\Lib\site-packages\torch\_dynamo\decorators.py)


tensor([6.8604, 4.9334, 3.3658])

In this example, we can avoid skipping by unrolling the loop:

In [4]:
@torch.compile
def fn(x):
    def inner(i):
        nonlocal x
        x = x + 1
        if i == 3:
            torch._dynamo.graph_break()
    inner(0)
    inner(1)
    inner(2)
    inner(3)
    inner(4)
    return x
fn(torch.randn(3))

torchdynamo start compiling fn C:\Users\Aditya\AppData\Local\Temp\ipykernel_4800\617960493.py:1, stack (elided 5 frames):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\dev\pytorch\venv\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\dev\pytorch\venv\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\dev\pytorch\venv\Lib\site-packages\ipykernel\kernelapp.py", line 758, in start
    self.io_loop.start()
  File "C:\dev\pytorch\venv\Lib\site-packages\tornado\platform\asyncio.py", line 211, in start
    self.asyncio_loop.run_forever()
  File "C:\Users\Aditya\AppData\Local\Programs\Python\Python313\Lib\asyncio\base_events.py", line 683, in run_forever
    self._run_once()
  File "C:\Users\Aditya\AppData\Local\Programs\Python\Python313\Lib\asyncio\base_events.py", line 2050, in _run_once
    handle._run()
  File

Step 1: torchdynamo start tracing fn C:\Users\Aditya\AppData\Local\Temp\ipykernel_4800\617960493.py:1


create_env


TRACE MAKE_CELL x []


TRACE starts_line C:\Users\Aditya\AppData\Local\Temp\ipykernel_4800\617960493.py:1 in fn
    @torch.compile


...


TREE_GUARD_MANAGER:
+- RootGuardManager
| +- LAMBDA_GUARD: torch._functorch.aot_autograd.utils.top_saved_tensors_hooks ids == None  # _dynamo\output_graph.py:688 in init_ambient_guards
| +- DEFAULT_DEVICE: utils_device.CURRENT_DEVICE == None                           # _dynamo\output_graph.py:676 in init_ambient_guards
| +- GLOBAL_STATE: ___check_global_state()
| +- TORCH_FUNCTION_MODE_STACK: ___check_torch_function_mode_stack()
| +- GuardManager: source=L['x'], accessed_by=FrameLocalsGuardAccessor(key='x', framelocals_idx=6), type=<class 'torch.Tensor'>, tag_safe=(True, False)
| | +- TENSOR_MATCH: check_tensor(L['x'], Tensor, DispatchKeySet(CPU, BackendSelect, ADInplaceOrView, AutogradCPU), torch.float32, device=None, requires_grad=False, size=[3], stride=[1])  # x = x + 1  # sers\Aditya\AppData\Local\Temp\ipykernel_4800\617960493.py:5 in inner
| | +- NO_HASATTR: hasattr(L['x'], '_dynamo_dynamic_indices') == False           # x = x + 1  # sers\Aditya\AppData\Local\Temp\ipykernel_4800

Guard eval latency = 264.50 us


put_code_state: no cache key, skipping


run_gc_after_compile: running gc


tensor([4.5721, 3.7991, 4.9834])

In general, resolving the graph break causing the skip will also resolve the skip.

## Graph Break in a Context Manager

Another common example of an unresumable graph break is a graph break in most context managers:

In [5]:
class CustomCtxManager:
    def __enter__(self):
        pass
    def __exit__(self, exc_type, exc_value, traceback):
        pass
@torch.compile
def fn(x):
    with CustomCtxManager():
        x = x + 1
        torch._dynamo.graph_break()
        return x + 1
fn(torch.randn(3))

torchdynamo start compiling fn C:\Users\Aditya\AppData\Local\Temp\ipykernel_4800\4148913404.py:6, stack (elided 5 frames):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\dev\pytorch\venv\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\dev\pytorch\venv\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\dev\pytorch\venv\Lib\site-packages\ipykernel\kernelapp.py", line 758, in start
    self.io_loop.start()
  File "C:\dev\pytorch\venv\Lib\site-packages\tornado\platform\asyncio.py", line 211, in start
    self.asyncio_loop.run_forever()
  File "C:\Users\Aditya\AppData\Local\Programs\Python\Python313\Lib\asyncio\base_events.py", line 683, in run_forever
    self._run_once()
  File "C:\Users\Aditya\AppData\Local\Programs\Python\Python313\Lib\asyncio\base_events.py", line 2050, in _run_once
    handle._run()
  Fil

Step 1: torchdynamo start tracing fn C:\Users\Aditya\AppData\Local\Temp\ipykernel_4800\4148913404.py:6


create_env


TRACE starts_line C:\Users\Aditya\AppData\Local\Temp\ipykernel_4800\4148913404.py:6 in fn
    @torch.compile


TRACE RESUME 0 []


...

Graph break: skip: from user code at:
  File "C:\Users\Aditya\AppData\Local\Temp\ipykernel_4800\4148913404.py", line 10, in fn
    torch._dynamo.graph_break()
Traceback (most recent call last):
  File "C:\dev\pytorch\venv\Lib\site-packages\torch\_dynamo\symbolic_convert.py", line 904, in wrapper
    return inner_fn(self, inst)
  File "C:\dev\pytorch\venv\Lib\site-packages\torch\_dynamo\symbolic_convert.py", line 3411, in CALL
    self._call(inst)
    ~~~~~~~~~~^^^^^^
  File "C:\dev\pytorch\venv\Lib\site-packages\torch\_dynamo\symbolic_convert.py", line 3405, in _call
    self.call_function(fn, args, kwargs)
    ~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^
  File "C:\dev\pytorch\venv\Lib\site-packages\torch\_dynamo\symbolic_convert.py", line 1266, in call_function
    self.push(fn.call_function(self, args, kwargs))  # type: ignore[arg-type]
              ~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^
  File "C:\dev\pytorch\venv\Lib\site-packages\torch\_dynamo\variables\lazy.py", line 212, in realize_and_

WON'T CONVERT fn C:\Users\Aditya\AppData\Local\Temp\ipykernel_4800\4148913404.py line 6 
due to: 
Traceback (most recent call last):
  File "C:\dev\pytorch\venv\Lib\site-packages\torch\_dynamo\symbolic_convert.py", line 904, in wrapper
    return inner_fn(self, inst)
  File "C:\dev\pytorch\venv\Lib\site-packages\torch\_dynamo\symbolic_convert.py", line 3411, in CALL
    self._call(inst)
    ~~~~~~~~~~^^^^^^
  File "C:\dev\pytorch\venv\Lib\site-packages\torch\_dynamo\symbolic_convert.py", line 3405, in _call
    self.call_function(fn, args, kwargs)
    ~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^
  File "C:\dev\pytorch\venv\Lib\site-packages\torch\_dynamo\symbolic_convert.py", line 1266, in call_function
    self.push(fn.call_function(self, args, kwargs))  # type: ignore[arg-type]
              ~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^
  File "C:\dev\pytorch\venv\Lib\site-packages\torch\_dynamo\variables\lazy.py", line 212, in realize_and_forward
    return getattr(self.realize(), name)(*args, **kwa

skipping because no torch.* __enter__             C:\Users\Aditya\AppData\Local\Temp\ipykernel_4800\4148913404.py 2


skipping because no torch.* __exit__             C:\Users\Aditya\AppData\Local\Temp\ipykernel_4800\4148913404.py 4


tensor([2.8117, 2.0776, 1.7796])

We can avoid skipping by moving the graph break outside of the context manager:

In [6]:
@torch.compile
def fn(x):
    with CustomCtxManager():
        x = x + 1
    torch._dynamo.graph_break()
    with CustomCtxManager():
        return x + 1
fn(torch.randn(3))

torchdynamo start compiling fn C:\Users\Aditya\AppData\Local\Temp\ipykernel_4800\2124425154.py:1, stack (elided 5 frames):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\dev\pytorch\venv\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\dev\pytorch\venv\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\dev\pytorch\venv\Lib\site-packages\ipykernel\kernelapp.py", line 758, in start
    self.io_loop.start()
  File "C:\dev\pytorch\venv\Lib\site-packages\tornado\platform\asyncio.py", line 211, in start
    self.asyncio_loop.run_forever()
  File "C:\Users\Aditya\AppData\Local\Programs\Python\Python313\Lib\asyncio\base_events.py", line 683, in run_forever
    self._run_once()
  File "C:\Users\Aditya\AppData\Local\Programs\Python\Python313\Lib\asyncio\base_events.py", line 2050, in _run_once
    handle._run()
  Fil

Step 1: torchdynamo start tracing fn C:\Users\Aditya\AppData\Local\Temp\ipykernel_4800\2124425154.py:1


create_env


TRACE starts_line C:\Users\Aditya\AppData\Local\Temp\ipykernel_4800\2124425154.py:1 in fn
    @torch.compile


TRACE RESUME 0 []


...


TREE_GUARD_MANAGER:
+- RootGuardManager
| +- LAMBDA_GUARD: torch._functorch.aot_autograd.utils.top_saved_tensors_hooks ids == None  # _dynamo\output_graph.py:688 in init_ambient_guards
| +- DEFAULT_DEVICE: utils_device.CURRENT_DEVICE == None                           # _dynamo\output_graph.py:676 in init_ambient_guards
| +- GLOBAL_STATE: ___check_global_state()
| +- TORCH_FUNCTION_MODE_STACK: ___check_torch_function_mode_stack()
| +- GuardManager: source=L['x'], accessed_by=FrameLocalsGuardAccessor(key='x', framelocals_idx=3), type=<class 'torch.Tensor'>, tag_safe=(True, False)
| | +- TENSOR_MATCH: check_tensor(L['x'], Tensor, DispatchKeySet(CPU, BackendSelect, ADInplaceOrView, AutogradCPU), torch.float32, device=None, requires_grad=False, size=[3], stride=[1])  # return x + 1  # sers\Aditya\AppData\Local\Temp\ipykernel_4800\2124425154.py:7 in torch_dynamo_resume_in_fn_at_5
| | +- NO_HASATTR: hasattr(L['x'], '_dynamo_dynamic_indices') == False           # return x + 1  # sers\Aditya\A

Guard eval latency = 77.20 us


put_code_state: no cache key, skipping


run_gc_after_compile: running gc


tensor([2.5893, 2.2229, 3.2566])

There are some context managers where Dynamo can resume after a graph break.
Some of these can be found in `supported_ctx_manager_classes` in `torch/_dynamo/variables/torch.py`.
In general, any context manager represented by a `ContextWrappingVariable` subclass in
`torch/_dynamo/variables/ctx_manager.py` support resuming after a graph break. For example:

In [7]:
import contextlib
@torch.compile
def fn(x):
    with contextlib.nullcontext():
        with torch.no_grad():
            x = x + 1
            torch._dynamo.graph_break()
            return x + 1
fn(torch.randn(3))

torchdynamo start compiling fn C:\Users\Aditya\AppData\Local\Temp\ipykernel_4800\3152636365.py:2, stack (elided 5 frames):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\dev\pytorch\venv\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\dev\pytorch\venv\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\dev\pytorch\venv\Lib\site-packages\ipykernel\kernelapp.py", line 758, in start
    self.io_loop.start()
  File "C:\dev\pytorch\venv\Lib\site-packages\tornado\platform\asyncio.py", line 211, in start
    self.asyncio_loop.run_forever()
  File "C:\Users\Aditya\AppData\Local\Programs\Python\Python313\Lib\asyncio\base_events.py", line 683, in run_forever
    self._run_once()
  File "C:\Users\Aditya\AppData\Local\Programs\Python\Python313\Lib\asyncio\base_events.py", line 2050, in _run_once
    handle._run()
  Fil

Step 1: torchdynamo start tracing fn C:\Users\Aditya\AppData\Local\Temp\ipykernel_4800\3152636365.py:2


create_env


TRACE starts_line C:\Users\Aditya\AppData\Local\Temp\ipykernel_4800\3152636365.py:2 in fn
    @torch.compile


TRACE RESUME 0 []


...


TREE_GUARD_MANAGER:
+- RootGuardManager
| +- LAMBDA_GUARD: torch._functorch.aot_autograd.utils.top_saved_tensors_hooks ids == None  # _dynamo\output_graph.py:688 in init_ambient_guards
| +- DEFAULT_DEVICE: utils_device.CURRENT_DEVICE == None                           # _dynamo\output_graph.py:676 in init_ambient_guards
| +- GLOBAL_STATE: ___check_global_state()
| +- TORCH_FUNCTION_MODE_STACK: ___check_torch_function_mode_stack()
| +- GuardManager: source=L['x'], accessed_by=FrameLocalsGuardAccessor(key='x', framelocals_idx=5), type=<class 'torch.Tensor'>, tag_safe=(True, False)
| | +- TENSOR_MATCH: check_tensor(L['x'], Tensor, DispatchKeySet(CPU, BackendSelect, ADInplaceOrView, AutogradCPU), torch.float32, device=None, requires_grad=False, size=[3], stride=[1])  # return x + 1  # sers\Aditya\AppData\Local\Temp\ipykernel_4800\3152636365.py:8 in torch_dynamo_resume_in_fn_at_7
| | +- NO_HASATTR: hasattr(L['x'], '_dynamo_dynamic_indices') == False           # return x + 1  # sers\Aditya\A

Guard eval latency = 51.50 us


put_code_state: no cache key, skipping


run_gc_after_compile: running gc


tensor([1.1801, 2.4112, 0.2910])

## Graph Break in a Try Block

A graph break in a try block cannot be resumed:

In [8]:
@torch.compile
def fn(x):
    try:
        x = x + 1
        torch._dynamo.graph_break()
        return x + 1
    except Exception as e:
        pass
fn(torch.randn(3))

torchdynamo start compiling fn C:\Users\Aditya\AppData\Local\Temp\ipykernel_4800\2546874632.py:1, stack (elided 5 frames):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\dev\pytorch\venv\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\dev\pytorch\venv\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\dev\pytorch\venv\Lib\site-packages\ipykernel\kernelapp.py", line 758, in start
    self.io_loop.start()
  File "C:\dev\pytorch\venv\Lib\site-packages\tornado\platform\asyncio.py", line 211, in start
    self.asyncio_loop.run_forever()
  File "C:\Users\Aditya\AppData\Local\Programs\Python\Python313\Lib\asyncio\base_events.py", line 683, in run_forever
    self._run_once()
  File "C:\Users\Aditya\AppData\Local\Programs\Python\Python313\Lib\asyncio\base_events.py", line 2050, in _run_once
    handle._run()
  Fil

Step 1: torchdynamo start tracing fn C:\Users\Aditya\AppData\Local\Temp\ipykernel_4800\2546874632.py:1


create_env


TRACE starts_line C:\Users\Aditya\AppData\Local\Temp\ipykernel_4800\2546874632.py:1 in fn
    @torch.compile


TRACE RESUME 0 []


...

empty checkpoint


run_gc_after_compile: running gc


Graph break: skip: from user code at:
  File "C:\Users\Aditya\AppData\Local\Temp\ipykernel_4800\2546874632.py", line 5, in fn
    torch._dynamo.graph_break()
Traceback (most recent call last):
  File "C:\dev\pytorch\venv\Lib\site-packages\torch\_dynamo\convert_frame.py", line 1624, in __call__
    result = self._inner_convert(
        frame, cache_entry, hooks, frame_state, skip=skip + 1
    )
  File "C:\dev\pytorch\venv\Lib\site-packages\torch\_dynamo\convert_frame.py", line 688, in __call__
    result = _compile(
        frame.f_code,
    ...<16 lines>...
        convert_frame_box=self._box,
    )
  File "C:\dev\pytorch\venv\Lib\site-packages\torch\_dynamo\convert_frame.py", line 1433, in _compile
    guarded_code, tracer_output = compile_inner(code, one_graph, hooks)
                                  ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\dev\pytorch\venv\Lib\site-packages\torch\_utils_internal.py", line 92, in wrapper_function
    return function(*args, **kwargs)
  File "

WON'T CONVERT fn C:\Users\Aditya\AppData\Local\Temp\ipykernel_4800\2546874632.py line 1 
due to: 
Traceback (most recent call last):
  File "C:\dev\pytorch\venv\Lib\site-packages\torch\_dynamo\convert_frame.py", line 1624, in __call__
    result = self._inner_convert(
        frame, cache_entry, hooks, frame_state, skip=skip + 1
    )
  File "C:\dev\pytorch\venv\Lib\site-packages\torch\_dynamo\convert_frame.py", line 688, in __call__
    result = _compile(
        frame.f_code,
    ...<16 lines>...
        convert_frame_box=self._box,
    )
  File "C:\dev\pytorch\venv\Lib\site-packages\torch\_dynamo\convert_frame.py", line 1433, in _compile
    guarded_code, tracer_output = compile_inner(code, one_graph, hooks)
                                  ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\dev\pytorch\venv\Lib\site-packages\torch\_utils_internal.py", line 92, in wrapper_function
    return function(*args, **kwargs)
  File "C:\dev\pytorch\venv\Lib\site-packages\torch\_dynamo\convert_

tensor([2.1518, 3.1957, 0.6976])

We can avoid skipping by moving the graph break outside of the try block:

In [9]:
@torch.compile
def fn(x):
    try:
        x = x + 1
    except Exception as e:
        pass
    torch._dynamo.graph_break()
    try:
        return x + 1
    except Exception as e:
        pass
fn(torch.randn(3))

torchdynamo start compiling fn C:\Users\Aditya\AppData\Local\Temp\ipykernel_4800\3015389759.py:1, stack (elided 5 frames):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\dev\pytorch\venv\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\dev\pytorch\venv\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\dev\pytorch\venv\Lib\site-packages\ipykernel\kernelapp.py", line 758, in start
    self.io_loop.start()
  File "C:\dev\pytorch\venv\Lib\site-packages\tornado\platform\asyncio.py", line 211, in start
    self.asyncio_loop.run_forever()
  File "C:\Users\Aditya\AppData\Local\Programs\Python\Python313\Lib\asyncio\base_events.py", line 683, in run_forever
    self._run_once()
  File "C:\Users\Aditya\AppData\Local\Programs\Python\Python313\Lib\asyncio\base_events.py", line 2050, in _run_once
    handle._run()
  Fil

Step 1: torchdynamo start tracing fn C:\Users\Aditya\AppData\Local\Temp\ipykernel_4800\3015389759.py:1


create_env


TRACE starts_line C:\Users\Aditya\AppData\Local\Temp\ipykernel_4800\3015389759.py:1 in fn
    @torch.compile


TRACE RESUME 0 []


...


TREE_GUARD_MANAGER:
+- RootGuardManager
| +- LAMBDA_GUARD: torch._functorch.aot_autograd.utils.top_saved_tensors_hooks ids == None  # _dynamo\output_graph.py:688 in init_ambient_guards
| +- DEFAULT_DEVICE: utils_device.CURRENT_DEVICE == None                           # _dynamo\output_graph.py:676 in init_ambient_guards
| +- GLOBAL_STATE: ___check_global_state()
| +- TORCH_FUNCTION_MODE_STACK: ___check_torch_function_mode_stack()
| +- GuardManager: source=L['x'], accessed_by=FrameLocalsGuardAccessor(key='x', framelocals_idx=3), type=<class 'torch.Tensor'>, tag_safe=(True, False)
| | +- TENSOR_MATCH: check_tensor(L['x'], Tensor, DispatchKeySet(CPU, BackendSelect, ADInplaceOrView, AutogradCPU), torch.float32, device=None, requires_grad=False, size=[3], stride=[1])  # return x + 1  # sers\Aditya\AppData\Local\Temp\ipykernel_4800\3015389759.py:9 in torch_dynamo_resume_in_fn_at_7
| | +- NO_HASATTR: hasattr(L['x'], '_dynamo_dynamic_indices') == False           # return x + 1  # sers\Aditya\A

Guard eval latency = 45.40 us


put_code_state: no cache key, skipping


run_gc_after_compile: running gc


tensor([1.7803, 1.7455, 0.9772])

## Hitting a Recompilation Limit
See [Changing the Cache Size Limit.](programming_model.recompilation.changing_cache_size_limit)

## Compiler Errors
Some compiler errors will result in skipped functions.
Other compiler errors will result in a hard error rather than a skipped function.

## Dealing with Skipped Functions
In general, you can resolve a skipped function by fixing the underlying graph break or error that
is causing the function to be skipped.

If the graph break/error causing the skipped function is difficult to fix,
then consider isolating the graph break/error in its own function so that minimal things are skipped.

In [10]:
def inner1(x):
    return x + 1
def inner2(x):
    return x + 2
@torch.compile
def fn(x):
    x = inner1(x)
    def problematic_code():
        torch._dynamo.skip_frame()
    problematic_code()
    x = inner2(x)
fn(torch.randn(3))

torchdynamo start compiling fn C:\Users\Aditya\AppData\Local\Temp\ipykernel_4800\273153676.py:5, stack (elided 5 frames):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\dev\pytorch\venv\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\dev\pytorch\venv\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\dev\pytorch\venv\Lib\site-packages\ipykernel\kernelapp.py", line 758, in start
    self.io_loop.start()
  File "C:\dev\pytorch\venv\Lib\site-packages\tornado\platform\asyncio.py", line 211, in start
    self.asyncio_loop.run_forever()
  File "C:\Users\Aditya\AppData\Local\Programs\Python\Python313\Lib\asyncio\base_events.py", line 683, in run_forever
    self._run_once()
  File "C:\Users\Aditya\AppData\Local\Programs\Python\Python313\Lib\asyncio\base_events.py", line 2050, in _run_once
    handle._run()
  File

Step 1: torchdynamo start tracing fn C:\Users\Aditya\AppData\Local\Temp\ipykernel_4800\273153676.py:5


create_env


TRACE starts_line C:\Users\Aditya\AppData\Local\Temp\ipykernel_4800\273153676.py:5 in fn
    @torch.compile


TRACE RESUME 0 []


...

GUARDS:



TREE_GUARD_MANAGER:
+- RootGuardManager
| +- LAMBDA_GUARD: torch._functorch.aot_autograd.utils.top_saved_tensors_hooks ids == None  # _dynamo\output_graph.py:688 in init_ambient_guards
| +- DEFAULT_DEVICE: utils_device.CURRENT_DEVICE == None                           # _dynamo\output_graph.py:676 in init_ambient_guards
| +- GLOBAL_STATE: ___check_global_state()
| +- TORCH_FUNCTION_MODE_STACK: ___check_torch_function_mode_stack()
| +- GuardManager: source=L['x'], accessed_by=FrameLocalsGuardAccessor(key='x', framelocals_idx=3), type=<class 'torch.Tensor'>, tag_safe=(True, False)
| | +- TENSOR_MATCH: check_tensor(L['x'], Tensor, DispatchKeySet(CPU, BackendSelect, ADInplaceOrView, AutogradCPU), torch.float32, device=None, requires_grad=False, size=[3], stride=[1])  # x = inner2(x)  # sers\Aditya\AppData\Local\Temp\ipykernel_4800\273153676.py:11 in torch_dynamo_resume_in_fn_at_10
| | +- NO_HASATTR: hasattr(L['x'], '_dynamo_dynamic_indices') == False           # x = inner2(x)  # sers\Adity

Guard eval latency = 52.00 us


put_code_state: no cache key, skipping


run_gc_after_compile: running gc
